# 第四章 在 LangChain 中调用 OpenAI Function Calling

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、Pydantic语法](#二、Pydantic语法)
     - [2.1 简单创建Python类](#2.1-简单创建Python类)
     - [2.2 使用Pydantic创建类](#2.2-使用Pydantic创建类)
 - [三、 基于Pydantic的OpenAI函数定义](#三、基于Pydantic的OpenAI函数定义)
     - [3.1 构造OpenAI的function](#3.1-构造OpenAI的function)
     - [3.2 通过langchain使用function](#3.2-通过langchain使用function)
     - [3.3 强制使用function](#3.3-强制使用function)
 - [四、使用function](#四、使用function)
     - [4.1 链式使用](#3.1-链式使用)
     - [4.2 使用多个function](#4.2-使用多个function)

为了大家实验方便，关于以下示例，我们将使用硅基流动的API，于是相关的代码也会修改。

# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、Pydantic语法

Pydantic 是 python 的“数据验证库”。
- 与 python 类型注释一起工作。但是，与静态类型检查不同，它们在运行时被积极地用于数据验证和转换。
- 提供内置方法来序列化/反序列化模型到 JSON ，字典等。
- LangChain 利用 Pydantic 创建 JsONScheme 描述函数。

## 2.1 简单创建Python类

在标准python中，可以创建一个`User`类，拥有`name`、`age`、`email`三种属性值。直接进行创建，不能对字段类型进行约束，即年龄中可能传入不合规的字符串类型。

In [1]:
from typing import List
from pydantic import BaseModel, Field

In [2]:
# 创建User类
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [3]:
# 生成user对象
foo = User(name="Joe",age=32, email="joe@gmail.com")

# 输出foo中 name字段
print(foo.name)

Joe


In [4]:
# 生成user对象
foo = User(name="Joe",age="bar", email="joe@gmail.com")

# 输出foo中 name字段
# name字段填写的是字符串类型，但能够创建成功并输出
print(foo.age)

bar


## 2.2 使用 Pydantic 创建类

使用 Pydantic 创建类，可以对类的属性值进行格式约束。在创建类的时候会进行格式验证，如果格式不符合要求会报错。

In [5]:
# 使用 Pydantic创建pUser类别，说明age使用int类型
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [6]:
# 生成user对象
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

# 输出foo中 name字段
print(foo_p.name)

Jane


In [7]:
# 使用了pydantic，由于age不是int，因此会报错。输出报错内容
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [8]:
class Class(BaseModel):
    students: List[pUser]

In [9]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# 三、基于Pydantic的OpenAI函数定义

## 3.1 构造 OpenAI 的 function

我们创建了一个`WeatherSearch`类，它继承自 Pydantic 的 BaseModel 子类.因此`WeatherSearch`类的所有成员都被具备了数据类型校验机制，该类有一个str类型的成员`airport_code`表示机场代码，并有一个描述信息description，用来说明 airport_code 的作用，在 airport_code 的上方也有一段文本描述信息：输入机场代码，获取该机场的天气信息。这段文本信息是对类`WeatherSearch`的说明，意思是通过机场代码可以查询天气情况。

同时，我们使用 langchain 将这个`WeatherSearch`类转换成openai的函数描述对象：

In [10]:
# 定义WeatherSearch
# WeatherSearch的function需要写注释完成函数的description
class WeatherSearch(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str = Field(description="输入机场代码查询天气")

In [13]:
# 导入langchain
from langchain_core.utils.function_calling import convert_to_openai_function

In [14]:
# 转化为openai function calling模式
weather_function = convert_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': '输入机场代码，获取该机场的天气信息',
 'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}



接下来，使用 langchain 的`convert_pydantic_to_openai_function`方法将 Pydantic 类转换成了 openai 的函数描述对象。需要的注意的是在定义Pydantic类时文本描述信息不可缺少，如缺少文本描述信息会导致转换时出错。

- `WeatherSearch1`，由于我们没有在`WeatherSearch1`中加入对本身的描述信息，导致在转换时报错。
- `WeatherSearch2`加入对本身的描述信息，因此不会报错。

In [16]:
# WeatherSearch1没有写注释，会报错
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="输入机场代码查询天气")

convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [17]:
# 构造WeatherSearch2，不对参数注释
class WeatherSearch2(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str

In [18]:
convert_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': '输入机场代码，获取该机场的天气信息',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

## 3.2 通过langchain使用function

为了实现 LLM 对 function 的调用，有一下两种方式

- 在`invoke`中指定 functions
- 执行`invoke`之前使用`bind`方法来绑定函数描述对象

In [22]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

In [23]:
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        )

In [25]:
# 在invoke中使用openai function功能

tools = [
    {
        "type": "function",
        "function": weather_function
    }
]

model.invoke("今天旧金山的天气怎么样？", tools=tools)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f2d0a1004306ca3311d0e6fee4d', 'function': {'arguments': ' {"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 177, 'total_tokens': 186, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f2d094af8bf93b2ad68d8b8e316', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f2c0d36a-fdde-4a51-9c89-94d59fa29529-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '01975f2d0a1004306ca3311d0e6fee4d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 9, 'total_tokens': 186, 'input_token_details': {}, 'output_token_details': {}})

In [26]:
# 直接在构造模型中声明functions，对话时不需要在声明
model_with_function = model.bind(tools=tools)
model_with_function.invoke("今天旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f2d3e53d6105e3f2634b5646b3a', 'function': {'arguments': ' {"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 177, 'total_tokens': 186, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f2d3d0e7ff69157e88c9093f187', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2aebffe0-625a-4993-a0e8-ffbc39790c1f-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '01975f2d3e53d6105e3f2634b5646b3a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 9, 'total_tokens': 186, 'input_token_details': {}, 'output_token_details': {}})

## 3.3 强制使用function

如果想要强制使用 function，需要在`bind`中增加`function_call`参数。

In [29]:
# 强制使用function，在模型构建时声明function_call
tool_choice={
    "type": "function",
    "function": {
     "name": "WeatherSearch"
    }
}

model_with_forced_function = model.bind(tools=tools, tool_choice=tool_choice)

In [30]:
# 基于已经声明的function对话，能够调用function
model_with_forced_function.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f3033b2983a089d8a76792b2350', 'function': {'arguments': '<tool_call>\n{"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 176, 'total_tokens': 186, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f3032a0567e660eb97a86eb7c57', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--88d6461f-f6db-4f33-a470-63c3e3bf0635-0', invalid_tool_calls=[{'name': 'WeatherSearch', 'args': '<tool_call>\n{"airport_code": "SFO"}', 'id': '01975f3033b2983a089d8a76792b2350', 'error': 'Function WeatherSearch arguments:\n\n<tool_call>\n{"airport_code": "SFO"}\n\nare not valid JSON. Received JSONDecodeError Expecting value: line 1 column 1 (char 0)\nFor troubleshooting, visit: https://python.langchain.co

In [34]:
# 输入hi，强制使用function的模型，即使prompt与函数description无关也会命中
model_with_forced_function.invoke("你好!")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f3078f82fa453ce958703094f94', 'function': {'arguments': '你好！有什么我可以帮助你的吗？', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 172, 'total_tokens': 180, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f3078036256513daf5e13f0ed7c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--22d39a08-b383-4eef-b7f5-3d213c2bdb8d-0', invalid_tool_calls=[{'name': 'WeatherSearch', 'args': '你好！有什么我可以帮助你的吗？', 'id': '01975f3078f82fa453ce958703094f94', 'error': 'Function WeatherSearch arguments:\n\n你好！有什么我可以帮助你的吗？\n\nare not valid JSON. Received JSONDecodeError Expecting value: line 1 column 1 (char 0)\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE ', 'type': 

In [35]:
# 输入hi，未强制使用function的模型，prompt与函数description无关不会命中
model_with_function.invoke("你好!")

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 172, 'total_tokens': 180, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f30ad2543bf9db7a04b16d74400', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a05208f0-831b-4a73-be05-6e97d48b8f60-0', usage_metadata={'input_tokens': 172, 'output_tokens': 8, 'total_tokens': 180, 'input_token_details': {}, 'output_token_details': {}})

# 四、使用function

在一般情况下我们会使用 chain 来实现整个问答的流程，接下来我们通过创建 chain 来实现函数调用功能

## 4.1 链式使用

In [36]:
# 引入需要的环境
from langchain.prompts import ChatPromptTemplate

In [37]:
# 使用预定义模版创建聊天代码
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手"),
    ("user", "{input}")
])

In [38]:
# 使用prompt + model_with_function 组合
chain = prompt | model_with_function

In [39]:
# 创建命中function的问答
chain.invoke({"input": "今天旧金山的天气怎么样？"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f313b70c8cfad71ab37e3dff767', 'function': {'arguments': ' {"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 185, 'total_tokens': 194, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f313a959e292274bd1e4b94ad06', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d7326508-c281-409d-af7a-24e7e8fff1dd-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '01975f313b70c8cfad71ab37e3dff767', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 9, 'total_tokens': 194, 'input_token_details': {}, 'output_token_details': {}})

In [40]:
# 创建未命中function的问答
chain.invoke({"input": "你好!"})

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 180, 'total_tokens': 188, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f314468345ae73ff624221d97ba', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--53a22908-9d06-4448-be24-74c084533788-0', usage_metadata={'input_tokens': 180, 'output_tokens': 8, 'total_tokens': 188, 'input_token_details': {}, 'output_token_details': {}})

## 4.2 使用多个function

我们可以传递一组函数，让 LLM 根据问题上下文决定使用哪个函数。

In [41]:
# 创建 ArtistSearch function
class ArtistSearch(BaseModel):
    """调用此命令可以获得特定艺术家的歌曲名称"""
    artist_name: str = Field(description="要查的艺术家的名字")
    n: int = Field(description="number of results")

In [48]:
# 组装WeatherSearch和ArtistSearch 函数
tools = [
    {
        "type": "function",
        "function": convert_to_openai_function(WeatherSearch)
    },
    {
        "type": "function",
        "function": convert_to_openai_function(ArtistSearch)
    }
]

In [49]:
# 将两个function放入模型
model_with_functions = model.bind(tools=tools)

In [50]:
# 命中 WeatherSearch function
model_with_functions.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f333331383cf04c8d44b68b6260', 'function': {'arguments': ' {"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 272, 'total_tokens': 281, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f3331bc5b86ec2ce00e54d59659', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--46b7082f-adaa-436a-810c-80a557571c9c-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '01975f333331383cf04c8d44b68b6260', 'type': 'tool_call'}], usage_metadata={'input_tokens': 272, 'output_tokens': 9, 'total_tokens': 281, 'input_token_details': {}, 'output_token_details': {}})

In [51]:
# 命中 ArtistSearch function
model_with_functions.invoke("找到泰勒·斯威夫特的三首歌？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f33455f3b976e01d8d8fc304f9f', 'function': {'arguments': ' {"artist_name": "Taylor Swift", "n": 3}', 'name': 'ArtistSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 279, 'total_tokens': 294, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f3343a22001bc0227746b877140', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--eaae31c2-83ef-4faf-8b93-bb018fc4fc88-0', tool_calls=[{'name': 'ArtistSearch', 'args': {'artist_name': 'Taylor Swift', 'n': 3}, 'id': '01975f33455f3b976e01d8d8fc304f9f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 15, 'total_tokens': 294, 'input_token_details': {}, 'output_token_details': {}})

In [52]:
# 命中都未命中
model_with_functions.invoke("你好!")

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 268, 'total_tokens': 276, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f335c90f5c91fa62ff83a7933b1', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--271711f6-a5b8-40c4-b213-ff55931c4dc4-0', usage_metadata={'input_tokens': 268, 'output_tokens': 8, 'total_tokens': 276, 'input_token_details': {}, 'output_token_details': {}})

# 五、英文版提示

**3.1 构造OpenAI的function**

In [53]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [54]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [55]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

**4.1 链式使用**

In [56]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

**4.2 使用多个function**

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")